# Spectral simulation for a blazar

In [22]:
import numpy as np
import astropy.units as u
from astropy.coordinates import SkyCoord
from gammapy.irf import load_irf_dict_from_file
from gammapy.maps import WcsGeom, MapAxis
from gammapy.modeling.models import (
    PowerLawSpectralModel,
    PointSpatialModel,
    SkyModel,
    Models,
    FoVBackgroundModel,
    EBLAbsorptionNormSpectralModel
)
from gammapy.makers import MapDatasetMaker, SafeMaskMaker
from gammapy.modeling import Fit
from gammapy.data import Observation
from gammapy.datasets import MapDataset

from gammapy.catalog import SourceCatalog4FGL



In [23]:
catalog = SourceCatalog4FGL()
catalog.description

'LAT 14-year point source catalog'

In [24]:
source = catalog["PKS 2155-304"]

In [25]:
irfs = load_irf_dict_from_file("$GAMMAPY_DATA/cta-1dc/caldb/data/cta/1dc/bcf/South_z20_50h/irf_file.fits")

Invalid unit found in background table! Assuming (s-1 MeV-1 sr-1)


In [70]:
# Define the observation parameters (typically the observation duration and the pointing position):
livetime = 2.0 * u.h
pointing = source.position

In [71]:
energy_reco = MapAxis.from_energy_edges(
    np.logspace(-1.0, 2.0, 20), unit="TeV", name="energy", interp="log"
)
geom = WcsGeom.create(
    skydir=source.position,
    binsz=0.02,
    width=(3, 3),
    frame="icrs",
    axes=[energy_reco],
)
# It is usually useful to have a separate binning for the true energy axis
energy_true = MapAxis.from_energy_edges(
    np.logspace(-1.5, 1.5, 50), unit="TeV", name="energy_true", interp="log"
)

empty = MapDataset.create(
    geom, name="dataset-simu", energy_axis_true=energy_true
)

In [72]:
sky_model = source.sky_model()
int_model = sky_model.spectral_model
print(int_model)

LogParabolaSpectralModel

type    name     value         unit        error   min max frozen is_norm link prior
---- --------- ---------- -------------- --------- --- --- ------ ------- ---- -----
     amplitude 1.2591e-11 MeV-1 s-1 cm-2 1.317e-13 nan nan  False    True           
     reference 1.1610e+03            MeV 0.000e+00 nan nan   True   False           
         alpha 1.7733e+00                1.029e-02 nan nan  False   False           
          beta 4.1893e-02                3.743e-03 nan nan  False   False           


In [73]:
redshift = 0.116
ebl = EBLAbsorptionNormSpectralModel.read_builtin("dominguez", redshift=redshift)



In [74]:
spectral_model = ebl * int_model
spatial_model = sky_model.spatial_model
model_simu = SkyModel(spectral_model=spectral_model, 
                      spatial_model=spatial_model,
                      name="model-simu")


In [75]:
bkg_model = FoVBackgroundModel(dataset_name="dataset-simu")

In [76]:
# Create an in-memory observation
obs = Observation.create(pointing=pointing, livetime=livetime, irfs=irfs)
print(obs)


Observation

	obs id            : 0 
 	tstart            : 51544.00
	tstop             : 51544.08
	duration          : 7200.00 s
	pointing (icrs)   : 329.7 deg, -30.2 deg

	deadtime fraction : 0.0%



/home/atreyeesinha/anaconda3/envs/gammapy-1.2/lib/python3.10/site-packages/gammapy/data/observations.py:281: GammapyDeprecationWarning: Pointing will be required to be provided as FixedPointingInfo
  warnings.warn(


In [77]:
maker = MapDatasetMaker(selection=["exposure", "background", "psf", "edisp"])

maker_safe_mask = SafeMaskMaker(methods=["offset-max"], offset_max=4.0 * u.deg)

dataset = maker.run(empty, obs)
dataset = maker_safe_mask.run(dataset, obs)



In [78]:
models = Models([sky_model, bkg_model])

dataset.models = models
dataset.fake(42)
# Do a print on the dataset - there is now a counts maps
print(dataset)


MapDataset
----------

  Name                            : dataset-simu 

  Total counts                    : 57135 
  Total background counts         : 53048.26
  Total excess counts             : 4086.74

  Predicted counts                : 57227.72
  Predicted background counts     : 53048.26
  Predicted excess counts         : 4179.46

  Exposure min                    : 4.25e+07 m2 s
  Exposure max                    : 3.60e+10 m2 s

  Number of total bins            : 427500 
  Number of fit bins              : 427500 

  Fit statistic type              : cash
  Fit statistic value (-2 log(L)) : 189312.56

  Number of models                : 2 
  Number of parameters            : 9
  Number of free parameters       : 6

  Component 0: SkyModel
  
    Name                      : 4FGL J2158.8-3013
    Datasets names            : None
    Spectral model type       : LogParabolaSpectralModel
    Spatial  model type       : PointSpatialModel
    Temporal model type       : 
    Parame

In [79]:
dataset.write("pks2155.fits.gz", overwrite=True)